In [ ]:
import os, pymupdf, re, sys, pyperclip
import pandas as pd
from tqdm import tqdm
from thefuzz import fuzz, process
import openpyxl as xl
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.comments import Comment

from openpyxl.styles import Alignment, Color, PatternFill, \
                            Font, Border, GradientFill, Side

ROOT_PATH = os.path.abspath('..')
print(ROOT_PATH)
username = os.getenv('USERNAME')

if ROOT_PATH not in sys.path:
    sys.path.append(ROOT_PATH)

from base.utils.office import find_row_after_str, find_row_from_str_on_col
from utils.ansarada import handle_index_a

from openpyxl.styles import Alignment, Color, PatternFill, \
                            Font, Border, GradientFill, Side


path = rf".\20241106 CENTRAL TM728 TM736_2.xlsm"

wb = xl.open(path, data_only=True)

c:\Users\mnoori\Downloads


ModuleNotFoundError: No module named 'base'

In [34]:
def get_default_value(ws, colstr, row, val):
    ws_value = ws[f'{colstr}{row}'].value
    if ws_value is None:
        return val
    else:
        return ws_value
    
def float_def(ws, colstr, row, to_int = False):
    value = get_default_value(ws, colstr, row, 0)
    if to_int:
        return int(value)
    return float(value)

def str_def(ws, colstr, row):
    return str(get_default_value(ws, colstr, row, ''))

def get_sd(ws, colstr, row, val):
    ws_value = ws[f'{colstr}{row}'].value
    if isinstance(ws_value, str):
        return val
    elif ws_value is not None:
        return ws_value

In [89]:
ts = wb['TAMPER']
rs = wb['REGULATOR']

def append_to_ws(ws, row, col, data, ix, dtype: float = str):
    if not isinstance(col, str):
        col = get_column_letter(col)
    if dtype == str:
        value = str_def(ws, col, row)
    elif dtype == float:
        value = float_def(ws, col, row)
    elif dtype == int:
        value = float_def(ws, col, row, True)
    data[ix].append(value)

def cust_append(data, ix, value):
    data[ix].append(value)

out_columns = ['Worksite', 'Client Work Order', 'Completed', 'Code',
               'Cycle', 'Track Section', 'From [km]', 'To [km]', 'Quantity', 'Speed [km/h]', 'Rail Temp [°C]',
               'Line Pre', 'Level Pre', 'Line Post', 'Level Post', '∆ Line', '∆ Level', 'Line Improvement %',
               'Level Improvement %', 'L+R', 'Slu Left', 'Slu Right', 'Lift', 'R Worksite', 'R Completed', 'R Client Work Order',
               'R From [km]', 'R To [km]', 'R Quantity', 'R Code', 'R Cycle', 'R Site Transfer', 'R Daily Crew Travel',
               'R Machine Breakdown', 'R Client Delays', 'R Other Delays', 'R Waiting on Trains', 'R Machine Travel',
               'R Machine Working', 'R Prestart & Maintenance', 'R Total Hours', 'R Total Regulated', 'R Operators1',
               'R Operators2', 'R OD Start', 'R OD End', 'R Odometer', 'R Comments', 'T Comments']

t_columns = ['Worksite', 'Client Work Order', 'Completed', 'Code', 'Cycle', 'Track Section', 'From [km]',
             'To [km]', 'Quantity', 'Speed [km/h]', 'Rail Temp [°C]', 'Line Pre', 'Level Pre', 'Line Post',
             'Level Post', '∆ Line', '∆ Level', 'Line Improvement %', 'Level Improvement %', 'L+R', 'Slu Left', 'Slu Right', 'Lift']

r_columns = ['R Worksite', 'R Completed', 'R Client Work Order', 'R From [km]',
             'R Track Section', 'R To [km]', 'R Quantity', 'R Code', 'R Cycle']

c_columns = ['File Name', 'Tamper', 'Regulator', 'District', 'Date', 'Day', 'Shift',
        'OD Start', 'OD End', 'Odometer', 'Site Transfer', 'Daily Crew Travel', 'Machine Breakdown',
        'Client Delays', 'Other Delays', 'Waiting on Trains', 'Machine Travel', 'Machine Working',
        'Prestart & Maintenance', 'Total Hours', 'Stabled', 'Total KM', 'JH Supervisor',
        'JH Operators1', 'JH Operators2', 'Operator Hours', 'R Site Transfer',
        'R Daily Crew Travel', 'R Machine Breakdown', 'R Client Delays', 'R Other Delays',
        'R Waiting on Trains', 'R Machine Travel', 'R Machine Working',
        'R Prestart & Maintenance', 'R Total Hours', 'R Total Regulated', 'R Supervisor',
        'R Operators1', 'R Operators2', 'R Operator Hours', 'R OD Start', 'R OD End', 'R Odometer', 'R Comments', 'T Comments']

In [131]:
c_data = {column : [] for column in c_columns}
t_data = {column : [] for column in t_columns}
r_data = {column : [] for column in r_columns}

for row_ix in range(26, 36):
    comp_cell = ts.cell(row_ix, 2)

    if not comp_cell.value: break

    t_from =  float_def(ts, 'T', row_ix); t_to =  float_def(ts, 'Y', row_ix)

    line_pre = get_sd(ts, 'AT', row_ix, 1.8)
    level_pre = get_sd(ts, 'AW', row_ix, 1.8)

    line_post = get_sd(ts, 'AZ', row_ix, 1.2)
    level_post = get_sd(ts, 'BC', row_ix, 1.2)

    d_line = line_pre - line_post; d_level = level_pre - level_post

    lpr = str_def(ts, 'BF', row_ix)
    l, r = lpr.split('+')
    lift = float_def(ts, 'BI', row_ix)

    append_to_ws(ts, row_ix, 1, t_data, 'Worksite', int)
    append_to_ws(ts, row_ix, 3, t_data, 'Client Work Order')
    append_to_ws(ts, row_ix, 2, t_data, 'Completed')
    append_to_ws(ts, row_ix, 5, t_data, 'Code')
    append_to_ws(ts, row_ix, 6, t_data, 'Cycle')
    append_to_ws(ts, row_ix, 8, t_data, 'Track Section')
    append_to_ws(ts, row_ix, 'T', t_data, 'From [km]', float)
    append_to_ws(ts, row_ix, 'Y', t_data, 'To [km]', float)
    cust_append(t_data, 'Quantity', t_from - t_to)
    append_to_ws(ts, row_ix, 'AI', t_data, 'Speed [km/h]', float)
    append_to_ws(ts, row_ix, 'AN', t_data, 'Rail Temp [°C]', float)
    cust_append(t_data, 'Line Pre', line_pre)
    cust_append(t_data, 'Level Pre', level_pre)
    cust_append(t_data, 'Line Post', line_post)
    cust_append(t_data, 'Level Post', level_post)
    cust_append(t_data, '∆ Line', d_line)
    cust_append(t_data, '∆ Level', d_level)
    cust_append(t_data, 'Line Improvement %', d_line/line_pre)
    cust_append(t_data, 'Level Improvement %', d_level/level_pre)
    cust_append(t_data, 'L+R', lpr)
    cust_append(t_data, 'Slu Left', l)
    cust_append(t_data, 'Slu Right', r)
    cust_append(t_data, 'Lift', lift)

t_data = pd.DataFrame(t_data)

for row_ix in range(26, 36):
    comp_cell = rs.cell(row_ix, 2)
    if not comp_cell.value: break
    r_from =  float_def(rs, 'T', row_ix); r_to =  float_def(rs, 'Y', row_ix)
    append_to_ws(rs, row_ix, 1, r_data, 'R Worksite', int)
    append_to_ws(rs, row_ix, 3, r_data, 'R Client Work Order')
    append_to_ws(rs, row_ix, 2, r_data, 'R Completed')
    append_to_ws(rs, row_ix, 5, r_data, 'R Code')
    append_to_ws(rs, row_ix, 6, r_data, 'R Cycle')
    append_to_ws(rs, row_ix, 8, r_data, 'R Track Section')
    append_to_ws(rs, row_ix, 'T', r_data, 'R From [km]', float)
    append_to_ws(rs, row_ix, 'Y', r_data, 'R To [km]', float)
    cust_append(r_data, 'R Quantity', r_from - r_to)

r_data = pd.DataFrame(r_data)

t_date = ts['AA6'].value
t_dist = str_def(ts, 'F', 6)
t_mach = str_def(ts, 'BE', 6)
r_mach = str_def(rs, 'BE', 6)

t_mw_1 = float_def(ts, 'BI', 19); t_mw_2 = float_def(ts, 'BI', 20)
r_mw_1 = float_def(rs, 'BI', 19); r_mw_2 = float_def(rs, 'BI', 20)

if not t_mach and r_mw_2 > r_mw_1 and r_mw_2 > 0:
    day = False
elif t_mw_2 > t_mw_1:
    day = False
elif r_mw_2 > r_mw_1:
    day = False
else:
    day = True

day = 'DAY' if day else 'NIGHT'

od_1 = float_def(ts, 'E', 8, True); od_2 = float_def(ts, 'E', 10, True)

if not (od_1 and od_2):
    odd = 0
else:
    odd = abs(od_2 - od_1)

t_work = float_def(ts, 'BI', 19) + float_def(ts, 'BI', 20)
r_work = float_def(rs, 'BI', 19) + float_def(ts, 'BI', 20)

cust_append(c_data, 'File Name', f'{t_date:%Y%m%d} {t_dist} {t_mach} {r_mach}')
cust_append(c_data, 'Tamper', t_mach)
cust_append(c_data, 'Regulator', r_mach)
cust_append(c_data, 'District', t_dist)
cust_append(c_data, 'Date', t_date)
cust_append(c_data, 'Day', f'{t_date:%A}')
cust_append(c_data, 'Shift', day)
cust_append(c_data, 'OD Start', float_def(ts, 'E', 8, True))
cust_append(c_data, 'OD End', float_def(ts, 'E', 10, True))
cust_append(c_data, 'Odometer', odd)

t_times = [
    float_def(ts, 'BI', 12),
    float_def(ts, 'BI', 13),
    float_def(ts, 'BI', 21),
    float_def(ts, 'BI', 15),
    float_def(ts, 'BI', 16),
    float_def(ts, 'BI', 17),
    float_def(ts, 'BI', 18),
    t_work,
    float_def(ts, 'BI', 14)
]

append_to_ws(ts, 12, 'BI', c_data, 'Site Transfer', float)
append_to_ws(ts, 13, 'BI', c_data, 'Daily Crew Travel', float)
append_to_ws(ts, 21, 'BI', c_data, 'Machine Breakdown', float)
append_to_ws(ts, 15, 'BI', c_data, 'Client Delays', float)
append_to_ws(ts, 16, 'BI', c_data, 'Other Delays', float)
append_to_ws(ts, 17, 'BI', c_data, 'Waiting on Trains', float)
append_to_ws(ts, 18, 'BI', c_data, 'Machine Travel', float)
cust_append(c_data, 'Machine Working', t_times[7])
append_to_ws(ts, 14, 'BI', c_data, 'Prestart & Maintenance', float)
cust_append(c_data, 'Total Hours', sum(t_times[2:-1]))
append_to_ws(ts, 36, 'T', c_data, 'Stabled')
cust_append(c_data, 'Total KM', t_data.Quantity.sum())
append_to_ws(ts, 40, 'BB', c_data, 'JH Supervisor')
append_to_ws(ts, 37, 'BB', c_data, 'JH Operators1')
append_to_ws(ts, 38, 'BB', c_data, 'JH Operators2')
append_to_ws(ts, 39, 'BB', c_data, 'Operator Hours')

r_times = [
    float_def(rs, 'BI', 12),
    float_def(rs, 'BI', 13),
    float_def(rs, 'BI', 21),
    float_def(rs, 'BI', 15),
    float_def(rs, 'BI', 16),
    float_def(rs, 'BI', 17),
    float_def(rs, 'BI', 18),
    r_work,
    float_def(rs, 'BI', 14)
]

cust_append(c_data, 'R Site Transfer', r_times[0])
cust_append(c_data, 'R Daily Crew Travel', r_times[1])
cust_append(c_data, 'R Machine Breakdown', r_times[2])
cust_append(c_data, 'R Client Delays', r_times[3])
cust_append(c_data, 'R Other Delays', r_times[4])
cust_append(c_data, 'R Waiting on Trains', r_times[5])
cust_append(c_data, 'R Machine Travel', r_times[6])
cust_append(c_data, 'R Machine Working', r_times[7])
cust_append(c_data, 'R Prestart & Maintenance', r_times[8])
cust_append(c_data, 'R Total Hours', sum(r_times[2:-1]))
cust_append(c_data, 'R Total Regulated', r_data['R Quantity'].sum())

append_to_ws(rs, 40, 'BB', c_data, 'R Supervisor')
append_to_ws(rs, 37, 'BB', c_data, 'R Operators1')
append_to_ws(rs, 38, 'BB', c_data, 'R Operators2')
append_to_ws(rs, 39, 'BB', c_data, 'R Operator Hours')

od_1 = float_def(rs, 'E', 8, True); od_2 = float_def(rs, 'E', 10, True)

if not (od_1 and od_2):
    odd = 0
else:
    odd = abs(od_2 - od_1)

cust_append(c_data, 'R OD Start', od_1)
cust_append(c_data, 'R OD End', od_2)
cust_append(c_data, 'R Odometer', odd)

append_to_ws(rs, 37, 'E', c_data, 'R Comments')
append_to_ws(ts, 40, 'F', c_data, 'T Comments')

c_data = pd.DataFrame(c_data)

In [130]:
c_data

,File Name,Tamper,Regulator,District,Date,Day,Shift,OD Start,OD End,Odometer,...,R Total Regulated,R Supervisor,R Operators1,R Operators2,R Operator Hours,R OD Start,R OD End,R Odometer,R Comments,T Comments
0,20241106 CENTRAL TM728 NG TM736 NG,TM728 NG,TM736 NG,CENTRAL,2024-11-06,Wednesday,DAY,50790,50800,10,...,1.01,T Briggs,A Magill,,12,0,0,0,,Machine Worked in Rottnest and all jobs double...


In [132]:
df_combined = pd.concat([t_data.reset_index(drop=True), r_data.reset_index(drop=True)], axis=1)
df_combined = pd.concat([df_combined, pd.concat([c_data] * len(df_combined), axis = 0, ignore_index = True)], axis = 1)
df_combined['LOC'] = 'WA'

In [133]:
df_combined

,Worksite,Client Work Order,Completed,Code,Cycle,Track Section,From [km],To [km],Quantity,Speed [km/h],...,R Supervisor,R Operators1,R Operators2,R Operator Hours,R OD Start,R OD End,R Odometer,R Comments,T Comments,LOC
0,1,9905262,Yes,MS4,P1DP,01MLD,-0.342,-1.318,0.976,80.0,...,T Briggs,A Magill,,12,0,0,0,,Machine Worked in Rottnest and all jobs double...,WA
1,2,9905263,No,PCS,P1DP,34XOB,-1.573,-1.607,0.034,80.0,...,T Briggs,A Magill,,12,0,0,0,,Machine Worked in Rottnest and all jobs double...,WA


In [121]:
print('\n'.join(df_combined.columns))

Worksite
Client Work Order
Completed
Code
Cycle
Track Section
From [km]
To [km]
Quantity
Speed [km/h]
Rail Temp [°C]
Line Pre
Level Pre
Line Post
Level Post
∆ Line
∆ Level
Line Improvement %
Level Improvement %
L+R
Slu Left
Slu Right
Lift
R Worksite
R Completed
R Client Work Order
R From [km]
R Track Section
R To [km]
R Quantity
R Code
R Cycle
File Name
Tamper
Regulator
District
Date
Day
Shift
OD Start
OD End
Odometer
Site Transfer
Daily Crew Travel
Machine Breakdown
Client Delays
Other Delays
Waiting on Trains
Machine Travel
Machine Working
Prestart & Maintenance
Total Hours
Stabled
Total KM
JH Supervisor
JH Operators1
JH Operators2
Operator Hours
R Site Transfer
R Daily Crew Travel
R Machine Breakdown
R Client Delays
R Other Delays
R Waiting on Trains
R Machine Travel
R Machine Working
R Prestart & Maintenance
R Total Hours
R Total Regulated
R Supervisor
R Operators1
R Operators2
R Operator Hours
R OD Start
R OD End
R Odometer
R Comments
T Comments
LOC


In [ ]:
path = rf"C:\Users\{username}\John Holland Group\JHG-Vermillion Tirana - Documents\initiatives\029-nrd\QRT Data\NSW\JH 20240831 0400 11 N24RES147.xlsm"
wb = xl.open(path, data_only=True)


In [122]:
lines = pyperclip.paste()
lines = lines.splitlines()

In [124]:
new_lines = []
for line in lines:
    new_lines.append(line.upper().replace(' ', '_'))

In [127]:
pyperclip.copy('\n'.join(new_lines))